In [0]:
def error_cb(err):
    """ The error callback is used for generic client errors. These
        errors are generally to be considered informational as the client will
        automatically try to recover from all errors, and no extra action
        is typically required by the application.
        For this example however, we terminate the application if the client
        is unable to connect to any broker (_ALL_BROKERS_DOWN) and on
        authentication errors (_AUTHENTICATION). """

    print("Client error: {}".format(err))
    if err.code() == KafkaError._ALL_BROKERS_DOWN or \
       err.code() == KafkaError._AUTHENTICATION:
        # Any exception raised from this callback will be re-raised from the
        # triggering flush() or poll() call.
        raise KafkaException(err)


def acked(err, msg):
    """ 
        Error callback is used for generic issues for producer errors. 
        
        Parameters:
            err (err): Error flag.
            msg (str): Error message that was part of the callback.
    """
    if err is not None:
        print("Failed to deliver message: %s: %s" % (str(msg), str(err)))
    else:
        print("Message produced: %s" % (str(msg)))

In [0]:
from confluent_kafka import Consumer
from time import sleep
import uuid
from confluent_kafka import Producer, Consumer, KafkaError, KafkaException
import json
from confluent_kafka.admin import AdminClient, NewTopic

#KAFKA variables, Move to the OS variables or configuration
# This will work in local Jupiter Notebook, but in a databrick, hiding config.py is tougher. 
confluentClusterName = "stage3talent"
confluentBootstrapServers = "pkc-ldvmy.centralus.azure.confluent.cloud:9092"
confluentTopicName = "Group3Data"
schemaRegistryUrl = "https://psrc-gq7pv.westus2.azure.confluent.cloud"
confluentApiKey = "YHMHG7E54LJA55XZ"
confluentSecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"
confluentRegistryApiKey = "YHMHG7E54LJA55XZ"
confluentRegistrySecret = "/XYn+w3gHGMqpe9l0TWvA9FznMYNln2STI+dytyPqtZ9QktH0TbGXUqepEsJ/nR0"

In [0]:
# Create topic

# Create admin_client
admin_client = AdminClient({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(uuid.uuid1()),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

In [0]:
# Add topic
topic_list = []
topic_list.append(NewTopic(confluentTopicName, 1, 3))
admin_client.create_topics(topic_list)
futures = admin_client.create_topics(topic_list)
try:
    record_metadata = []
    for k, future in futures.items():
        # f = i.get(timeout=10)
        print(f"type(k): {type(k)}")
        print(f"type(v): {type(future)}")
        try:
            print(future.result())
        except:
            print("Topic already exists")
except KafkaError:
    # Decide what to do if produce request failed...
    print(traceback.format_exc())
    result = 'Fail'
finally:
    print("finally")

type(k): <class 'str'>
type(v): <class 'concurrent.futures._base.Future'>
Topic already exists
finally

In [0]:
#Create producer object

p = Producer({
    'bootstrap.servers': confluentBootstrapServers,
    'sasl.mechanism': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': confluentApiKey,
    'sasl.password': confluentSecret,
    'group.id': str(1),  # this will create a new consumer group on each invocation.
    'auto.offset.reset': 'earliest',
    'error_cb': error_cb,
})

In [0]:
%fs
ls /mnt/capstone-group3-raw/

path,name,size,modificationTime
dbfs:/mnt/capstone-group3-raw/ACSST5Y2020.S1901_dataset.csv,ACSST5Y2020.S1901_dataset.csv,1520328,1659984095000
dbfs:/mnt/capstone-group3-raw/DistrictDataCleaned.csv,DistrictDataCleaned.csv,48613,1659732238000
dbfs:/mnt/capstone-group3-raw/FederallyFundedDistricts.csv,FederallyFundedDistricts.csv,510945,1659754753000
dbfs:/mnt/capstone-group3-raw/NYCSchoolDistricts.csv,NYCSchoolDistricts.csv,347923,1659751742000
dbfs:/mnt/capstone-group3-raw/csvfiles/,csvfiles/,0,0
dbfs:/mnt/capstone-group3-raw/district-householdincome.csv,district-householdincome.csv,29882,1659730056000
dbfs:/mnt/capstone-group3-raw/districtdata.csv/,districtdata.csv/,0,1659751605000
dbfs:/mnt/capstone-group3-raw/districtdatafinal.csv,districtdatafinal.csv,50790,1660315269000
dbfs:/mnt/capstone-group3-raw/districtdatafinalcleaning.csv/,districtdatafinalcleaning.csv/,0,1659835886000
dbfs:/mnt/capstone-group3-raw/districtdatamedianincomerevised.csv/,districtdatamedianincomerevised.csv/,0,1659792537000


In [0]:
df = spark.read.option("header",True).csv('dbfs:/mnt/capstone-group3-raw/districtdatafinal.csv/')

In [0]:
display(df)

DistrictName,GraduationPercentageRate,DropoutPercentageRate,FundingPerStudent,TotalEnrolled,MalesEnrolled,FemalesEnrolled,Percentage of Minority Students,HouseholdIncome,FederallyFunded
YORKTOWN CSD,97.0,1.0,24424.21,3401.0,1775.0,1626.0,27.8,115881.16,No
CARMEL CSD,92.0,3.0,27298.65,3979.0,2034.0,1945.0,37.6,110606.5,No
CARTHAGE CSD,89.0,5.0,17149.15,3065.0,1602.0,1463.0,25.9,53290.64,Yes
CASSADAGA VALLEY CSD,88.0,7.0,20862.54,799.0,422.0,377.0,6.0,55387.63,null
CATO-MERIDIAN CSD,89.0,5.0,18088.09,860.0,449.0,411.0,8.1,64765.75,null
CATSKILL CSD,91.0,4.0,24088.35,1350.0,668.0,682.0,32.9,58658.83,null
CATTARAUGUS-LITTLE VALLEY CSD,93.0,4.0,19362.54,858.0,431.0,427.0,6.9,52411.0,null
CAZENOVIA CSD,94.0,2.0,17708.25,1360.0,727.0,633.0,6.0,69543.88,null
CENTER MORICHES UFSD,93.0,1.0,24593.36,1513.0,788.0,725.0,26.8,99102.0,null
CENTRAL ISLIP UFSD,88.0,1.0,21656.92,7571.0,3929.0,3642.0,96.1,101402.4,null


In [0]:
from time import sleep

fulldistrictinfo = []
columns = []

for column in df.columns:
    columns.append(column)

    
for row in df.rdd.collect():
    
    districtdict = {}

    for index in range(len(df.columns)):

        districtdict[df.columns[index]] = row[index]
        
    fulldistrictinfo.append(districtdict)
    

for eachrow in range(len(fulldistrictinfo)):
    
    district = fulldistrictinfo[eachrow]
    p.produce(confluentTopicName,json.dumps(district))
    p.flush()
    sleep(2)
    print(district)
    
    
    
    
 

{'DistrictName': 'YORKTOWN CSD', 'GraduationPercentageRate': '97.0', 'DropoutPercentageRate': '1.0', 'FundingPerStudent': '24424.21', 'TotalEnrolled': '3401.0', 'MalesEnrolled': '1775.0', 'FemalesEnrolled': '1626.0', 'Percentage of Minority Students': '27.8', 'HouseholdIncome': '115881.16', 'FederallyFunded': 'No'}
{'DistrictName': 'CARMEL CSD', 'GraduationPercentageRate': '92.0', 'DropoutPercentageRate': '3.0', 'FundingPerStudent': '27298.65', 'TotalEnrolled': '3979.0', 'MalesEnrolled': '2034.0', 'FemalesEnrolled': '1945.0', 'Percentage of Minority Students': '37.6', 'HouseholdIncome': '110606.5', 'FederallyFunded': 'No'}
{'DistrictName': 'CARTHAGE CSD', 'GraduationPercentageRate': '89.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '17149.15', 'TotalEnrolled': '3065.0', 'MalesEnrolled': '1602.0', 'FemalesEnrolled': '1463.0', 'Percentage of Minority Students': '25.9', 'HouseholdIncome': '53290.64', 'FederallyFunded': 'Yes'}
{'DistrictName': 'CASSADAGA VALLEY CSD', 'GraduationPercentageRate': '88.0', 'DropoutPercentageRate': '7.0', 'FundingPerStudent': '20862.54', 'TotalEnrolled': '799.0', 'MalesEnrolled': '422.0', 'FemalesEnrolled': '377.0', 'Percentage of Minority Students': '6.0', 'HouseholdIncome': '55387.63', 'FederallyFunded': None}
{'DistrictName': 'CATO-MERIDIAN CSD', 'GraduationPercentageRate': '89.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '18088.09', 'TotalEnrolled': '860.0', 'MalesEnrolled': '449.0', 'FemalesEnrolled': '411.0', 'Percentage of Minority Students': '8.1', 'HouseholdIncome': '64765.75', 'FederallyFunded': None}
{'DistrictName': 'CATSKILL CSD', 'GraduationPercentageRate': '91.0', 'DropoutPercentageRate': '4.0', 'FundingPerStudent': '24088.35', 'TotalEnrolled': '1350.0', 'MalesEnrolled': '668.0', 'FemalesEnrolled': '682.0', 'Percentage of Minority Students': '32.9', 'HouseholdIncome': '58658.83', 'FederallyFunded': None}
{'DistrictName': 'CATTARAUGUS-LITTLE VALLEY CSD', 'GraduationPercentageRate': '93.0', 'DropoutPercentageRate': '4.0', 'FundingPerStudent': '19362.54', 'TotalEnrolled': '858.0', 'MalesEnrolled': '431.0', 'FemalesEnrolled': '427.0', 'Percentage of Minority Students': '6.9', 'HouseholdIncome': '52411.0', 'FederallyFunded': None}
{'DistrictName': 'CAZENOVIA CSD', 'GraduationPercentageRate': '94.0', 'DropoutPercentageRate': '2.0', 'FundingPerStudent': '17708.25', 'TotalEnrolled': '1360.0', 'MalesEnrolled': '727.0', 'FemalesEnrolled': '633.0', 'Percentage of Minority Students': '6.0', 'HouseholdIncome': '69543.88', 'FederallyFunded': None}
{'DistrictName': 'CENTER MORICHES UFSD', 'GraduationPercentageRate': '93.0', 'DropoutPercentageRate': '1.0', 'FundingPerStudent': '24593.36', 'TotalEnrolled': '1513.0', 'MalesEnrolled': '788.0', 'FemalesEnrolled': '725.0', 'Percentage of Minority Students': '26.8', 'HouseholdIncome': '99102.0', 'FederallyFunded': None}
{'DistrictName': 'CENTRAL ISLIP UFSD', 'GraduationPercentageRate': '88.0', 'DropoutPercentageRate': '1.0', 'FundingPerStudent': '21656.92', 'TotalEnrolled': '7571.0', 'MalesEnrolled': '3929.0', 'FemalesEnrolled': '3642.0', 'Percentage of Minority Students': '96.1', 'HouseholdIncome': '101402.4', 'FederallyFunded': None}
{'DistrictName': 'CENTRAL SQUARE CSD', 'GraduationPercentageRate': '87.0', 'DropoutPercentageRate': '8.0', 'FundingPerStudent': '17310.54', 'TotalEnrolled': '3580.0', 'MalesEnrolled': '1827.0', 'FemalesEnrolled': '1753.0', 'Percentage of Minority Students': '6.1', 'HouseholdIncome': '67685.73', 'FederallyFunded': None}
{'DistrictName': 'CENTRAL VALLEY CSD AT ILION-MOHAWK', 'GraduationPercentageRate': '87.0', 'DropoutPercentageRate': '5.0', 'FundingPerStudent': '15522.11', 'TotalEnrolled': '2082.0', 'MalesEnrolled': '1025.0', 'FemalesEnrolled': '1057.0', 'Percentage of Minority Students': '9.4', 'HouseholdIncome': None, 'FederallyFunded': None}
{'DistrictName': 'CHAPPAQUA CSD', 'GraduationPercentageRate': '99.0', 'DropoutPercentageRate': '0.0', 'FundingPerStudent': '29069.23', 'TotalEnrolled': '3625.0', 'M